In [23]:
import os
from mistralai import Mistral
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import cv2
from dotenv import load_dotenv

In [29]:
def generate_invoice_image(filepath):
    images = convert_from_path(filepath)
    filename = os.path.splitext(os.path.split(filepath)[1])[0]
    
    for i, img in enumerate(images):
        if not os.path.isdir('output_images/'):
            os.mkdir('output_images')
        img_path = f"output_images/{filename}.jpg"
        img.save(img_path, "PNG")
        print(f"Saved: {img_path}")

path = './pdfs/invc_2[1].pdf'
image = generate_invoice_image(path)


Saved: output_images/invc_2[1].jpg


In [30]:
image_path = './output_images/invc_2[1].jpg'

In [17]:
def ProcessImage(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # De-noising
    denoised = cv2.fastNlMeansDenoising(gray_img, h=10)
    # Binary Thresholding
    _, thresh = cv2.threshold(denoised, 240, 255, cv2.THRESH_BINARY)
    # Adaptive Thresholding
    thresh1 = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 15, 2)
    thresh1_1 = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 55, 5)
    thresh2 = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 55, 5)
    
    cv2.namedWindow("Thresh1_1", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Thresh1_1", 800, 600)
    cv2.imshow("Thresh1_1", thresh1_1)
    # cv2.imshow('Blurred', thresh1)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    cv2.imwrite('./output_images/invoice_image_processed.jpg', thresh1_1)
    
    return './output_images/invoice_image_processed.jpg'
    


In [83]:
def invoice_image_text_extract(img_path):
    invoice_text = pytesseract.image_to_string(Image.open(img_path))
    return invoice_text

processed_img_path = ProcessImage(image_path)
invoice_text = invoice_image_text_extract(processed_img_path)
print(invoice_text)

error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'


In [31]:
from paddleocr import PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang="en")

[2025/03/05 23:49:47] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\ADMIN/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\ADMIN/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=

In [32]:
def merge_ocr_results(ocr_results, y_threshold=20):

    # Sort OCR results
    ocr_results.sort(key=lambda item: (item[0][0][1], item[0][0][0]))

    # print("*************Debugging***********")
    # for idx in range(len(ocr_results)):
    #     res = ocr_results[idx]
    #     for line in res:
    #         print(line)

    merged_result = []
    for idx in range(len(ocr_results)):
        res = ocr_results[idx]
        for box, (text, conf) in res:
            _, y1 = box[0]
            # x2, _ = box[1]

            if not merged_result:
                merged_result.append([box, (text, conf)])
                continue

            prev_box, (prev_text, prev_conf) = merged_result[-1]
            _, prev_y1 = prev_box[0]

            # print(f"DEBUG : x1 - prev_x1 = {x1 - prev_x1}")
            # print(f"DEBUG : y1 - prev_y1 = {y1 - prev_y1}")

            if abs(y1 - prev_y1) < y_threshold:
                merged_txt = prev_text + " " + text
                avg_conf = (conf + prev_conf) / 2
                prev_box[1], prev_box[2] = box[1], box[2]
                merged_result[-1] = [prev_box, (merged_txt, avg_conf)]
            else:
                merged_result.append([box, (text, conf)])

    return merged_result

In [33]:
result = ocr.ocr(image_path, cls=True)
# merged_result = merge_ocr_results(result)
text = ""
for idx in range(len(result)):
    res = result[idx]
    for line in res:
        text += line[1][0] + '\n'

# for res in merged_result:
#     print(res)

[2025/03/05 23:49:56] ppocr DEBUG: dt_boxes num : 308, elapsed : 1.0992214679718018
[2025/03/05 23:49:59] ppocr DEBUG: cls num  : 308, elapsed : 2.0799665451049805
[2025/03/05 23:52:41] ppocr DEBUG: rec_res num  : 308, elapsed : 162.62366199493408


In [34]:
print(text)

P.O No.23126.0.0063DATE.Q3073
NREADS
RECEIPTNO.O.DATE-T3
B.P.VNo.
COMPO
BPv:23o1\o18y
SIGNATURE,
Invoice
NO : 1013
02-06-2023
M/S; Integrated quality business LLC
SI No
Name
LOCCompST DateE Date
Section Days
Rate
Total
Adbul Hak
SLL
Octal
5-1-23
5-31-23
Helper
31
175.000
1
175.000
2
Al Amin
SLI
Octal
5-1-23
5-31-23
Helper
31
175.000
175.000
3
Al Toufiq
SLL
Octal
5-1-23
5-31-23
Helper
31
175.000
175.000
4
Anusal
SLL
Octal
5-1-23
531-23
[Helper
175.000
175.000
31
5
Anwarul Hussain
77S
oeel
5-1-23
5-31-23
Helper
31
175.000
175.000
6
Ariful Islam
115
5-1-23
5-31-23|Helper
31
175.000
175.000
Asik rahman
SLL
Octal
5-1-23
5-31-23
Helper
31
175.000
175.000
R
Jabad Mia
SLL.
Octal
5-1-23
5-31-23
Helper
31
175.000
175.000
Jamil Rahman
SLL
Octal
51-23
5-31-23
Helper
31
175.000
175.000
10
Jowel md
SLL.
Octal
5-1-23
5-31-23
Helper
31
175.000
175.000
11
Joynal
SLL
Octal
5-1-23
5-31-23
Helper
31
175.000
175.000
12
Jutan C Das
SLL
Octal
5-1-23
5-31-23 Helper
31
175.000
175.000
13
Kasi Adbul Kader
SLL
O

In [38]:
def analyze_text(text):
    model = "mistral-large-latest"
    prompt = f"Analyze the given text from an invoice and extract the following information \nInvoice Number, Date, Total Amount and Organization Name\nReturn the extracted information on new lines in the form fieldname : value \nBe careful with Invoice Number since it can also be labeled as Document Number in the document. Do not add any other text in the response other than the required fields\n{text}"
    
    load_dotenv("C:\\Users\\ADMIN\\Desktop\\api_key.env")
    mistralai_api_key = os.getenv("MISTRALAI_API_KEY")
    
    client = Mistral(api_key=mistralai_api_key)
    
    completion = client.chat.complete(
        model=model,
        messages=[
            {"role":"user", "content":prompt}
        ]
    )
    
    return completion.choices[0].message.content
    

In [39]:
analyzed_text = analyze_text(text)
print(analyzed_text)

Invoice Number : 1013
Date : 02-06-2023
Total Amount : 5250.000
Organization Name : Integrated quality business LLC
